In [1]:
# from haystack.telemetry import tutorial_running

# tutorial_running(27)

In [2]:
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from datasets import load_dataset
from haystack import Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
import os
from getpass import getpass

from haystack.components.generators import HuggingFaceLocalGenerator
from haystack import Pipeline
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack import Document
import os
from pathlib import Path
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever, ChromaEmbeddingRetriever
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner


/home/atharva/anaconda3/envs/pytorchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/atharva/anaconda3/envs/pytorchEnv/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
document_store = ChromaDocumentStore()
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="sentence", split_length=1)
# document_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
# docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
# print(dataset)
# print(docs)

HERE = Path.cwd()
file_paths = [HERE / "data" / Path(name) for name in os.listdir("data")]
print(file_paths)

[PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_28.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_08.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_26.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_06.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_11.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_10.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_20.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_45.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_46.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_05.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_25.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_32.txt'), PosixPath('/home/atharva/Atharva/webapp/documentRetrieval/data/usr_03.txt')

In [4]:

# doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
# doc_embedder.warm_up()

In [5]:
# docs_with_embeddings = doc_embedder.run(docs)
# document_store.write_documents(docs_with_embeddings["documents"])

In [6]:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

converter = TextFileToDocument()
docs = []
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
        doc = Document(content=content, meta={"name": file_path.name})
        
        docs.append(doc)
# print(type(docs))
# print(docs[0])
print(docs[0])
docs = document_cleaner.run(docs)
print(docs["documents"][0])
docs = document_splitter.run(docs["documents"])
print(docs["documents"][0])
# print(type(docs))
# print(docs)
docs_with_embeddings = doc_embedder.run(docs["documents"])
document_store.write_documents(docs_with_embeddings["documents"])

# # Create an indexing pipeline
# indexing = Pipeline()
# indexing.add_component("converter", TextFileToDocument())
# # indexing.connect("converter", "converter")

# # Convert the text files to documents
# docs = indexing.run({"converter": {"sources": file_paths}})["converter"]["documents"]

# # Embed the documents
# docs_with_embeddings = doc_embedder.run(docs)

# # Write the documents with embeddings to the document store
# document_store.write_documents(docs_with_embeddings["documents"])

/home/atharva/anaconda3/envs/pytorchEnv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Document(id=2fd892750b565f1b268b9dc90934b87d65d17d842b5f1151feaf6bc257100dc8, content: '*usr_28.txt*	For Vim version 9.0.  Last change: 2008 Jun 14

		     VIM USER MANUAL - by Bram Moolen...', meta: {'name': 'usr_28.txt'})
Document(id=9116d88ef6e2632c599f0de1aada539e862a00d05c6d3d2abdc785c1a92ff9dc, content: '*usr_28.txt*	For Vim version 9.0. Last change: 2008 Jun 14 VIM USER MANUAL - by Bram Moolenaar Foldi...', meta: {'name': 'usr_28.txt'})
Document(id=3520de7f1f911cf5e99d591f096a68fc134a2c8de8ab9ccebb04562d3608c51b, content: '*usr_28.', meta: {'name': 'usr_28.txt', 'source_id': '9116d88ef6e2632c599f0de1aada539e862a00d05c6d3d2abdc785c1a92ff9dc', 'page_number': 1})


Batches: 100%|██████████| 264/264 [00:36<00:00,  7.15it/s]
Insert of existing embedding ID: 47048f33769f4eeeaab21bada5b51005220fd9e66344836d2e28e305fbd65658
Add of existing embedding ID: 47048f33769f4eeeaab21bada5b51005220fd9e66344836d2e28e305fbd65658
Insert of existing embedding ID: 47048f33769f4eeeaab21bada5b51005220fd9e66344836d2e28e305fbd65658
Add of existing embedding ID: 47048f33769f4eeeaab21bada5b51005220fd9e66344836d2e28e305fbd65658
Insert of existing embedding ID: 79488db9e4bb007146e3c2d3da3cdbdfa931881d6cd5a23f0cf493f841e44388
Add of existing embedding ID: 79488db9e4bb007146e3c2d3da3cdbdfa931881d6cd5a23f0cf493f841e44388
Insert of existing embedding ID: 79488db9e4bb007146e3c2d3da3cdbdfa931881d6cd5a23f0cf493f841e44388
Add of existing embedding ID: 79488db9e4bb007146e3c2d3da3cdbdfa931881d6cd5a23f0cf493f841e44388
Insert of existing embedding ID: 79488db9e4bb007146e3c2d3da3cdbdfa931881d6cd5a23f0cf493f841e44388
Add of existing embedding ID: 79488db9e4bb007146e3c2d3da3cdbdfa931881d6

8434

In [7]:
# Create a querying pipeline
# querying = Pipeline()
# querying.add_component("retriever", ChromaQueryTextRetriever(document_store))
# results = querying.run({"retriever": {"query": "Variable declarations", "top_k": 2}})
# print((querying))
retriever = ChromaEmbeddingRetriever(document_store)
print(retriever)
# # Display the results
# for d in results["retriever"]["documents"]:
#     print(d.meta, d.score)

Inputs:
  - query_embedding: List[float]
  - filters: Optional[Dict[str, Any]]
  - top_k: Optional[int]
Outputs:
  - documents: List[Document]


In [8]:
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
print(text_embedder)

Inputs:
  - text: str
Outputs:
  - embedding: List[float]


In [9]:

# retriever = InMemoryEmbeddingRetriever(document_store)


In [10]:

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

In [11]:


generator = HuggingFaceLocalGenerator(model="google/flan-t5-small",
                                      task="text2text-generation",
                                      generation_kwargs={
                                        "max_new_tokens": 100,
                                        # "temperature": 0.9,
                                        })

# generator = HuggingFaceLocalGenerator(model="malmarjeh/bert2bert",
#                                       task="text2text-generation",
#                                       generation_kwargs={
#                                         "max_new_tokens": 100,
#                                         # "temperature": 0.9,
#                                         })

# generator = HuggingFaceLocalGenerator(model="openai-community/gpt2",
#                                       task="text-generation",
#                                       generation_kwargs={
#                                         "max_new_tokens": 50,
#                                         # "temperature": 0.9,
#                                         })

generator.warm_up()

# reader = TransformersReader(model_name_or_path="bert-large-uncased-whole-word-masking-finetuned-squad")


In [12]:


basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: ChromaEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [13]:
question = "How to suspend vim?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Batches: 100%|██████████| 1/1 [00:00<00:00, 86.46it/s]


Press CTRL-Z


In [14]:
print(response["llm"]["replies"][0])

Press CTRL-Z


In [15]:
print(response["llm"]["replies"])

['Press CTRL-Z']


In [16]:
"""
"Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
"""

'\n"Where is Gardens of Babylon?",\n    "Why did people build Great Pyramid of Giza?",\n    "What does Rhodes Statue look like?",\n    "Why did people visit the Temple of Artemis?",\n    "What is the importance of Colossus of Rhodes?",\n    "What happened to the Tomb of Mausolus?",\n    "How did Colossus of Rhodes collapse?",\n'